In [2]:
%load_ext rpy2.ipython
import rpy2.robjects.lib.ggplot2 as ggplot2
import childespy
import numpy as np
import os
import imp
import pandas as pd
import Levenshtein
import itertools
import re

/Users/nicolewong/opt/anaconda3/envs/child_grammar/lib/python3.10/site-packages/rpy2/robjects/lib/ggplot2.py:70: UserWarning: This was designed againt ggplot2 versions starting with 3.3. but you have 3.4.0
  warnings.warn(


Reinstalling childesr version 0.2.1
--- Please select a CRAN mirror for use in this session ---
Secure CRAN mirrors 

 1: 0-Cloud [https]
 2: Australia (Canberra) [https]
 3: Australia (Melbourne 1) [https]
 4: Australia (Melbourne 2) [https]
 5: Australia (Perth) [https]
 6: Austria [https]
 7: Belgium (Brussels) [https]
 8: Brazil (PR) [https]
 9: Brazil (RJ) [https]
10: Brazil (SP 1) [https]
11: Brazil (SP 2) [https]
12: Bulgaria [https]
13: Canada (MB) [https]
14: Canada (ON 3) [https]
15: Chile (Santiago) [https]
16: China (Beijing 2) [https]
17: China (Beijing 3) [https]
18: China (Hefei) [https]
19: China (Hong Kong) [https]
20: China (Guangzhou) [https]
21: China (Lanzhou) [https]
22: China (Nanjing) [https]
23: China (Shanghai 2) [https]
24: China (Shenzhen) [https]
25: Colombia (Cali) [https]
26: Costa Rica [https]
27: Cyprus [https]
28: Czech Republic [https]
29: Denmark [https]
30: East Asia [https]
31: Ecuador (Cuenca) [https]
32: Ecuador (Quito) [https]
33: France (Lyon 1

R[write to console]: trying URL 'https://cran.case.edu/bin/macosx/contrib/4.2/childesr_0.2.3.tgz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 75307 bytes (73 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to con


The downloaded binary packages are in
	/var/folders/g7/6fmhhq2x5813j_xlfxrqxln80000gn/T//Rtmpxo1EKW/downloaded_packages


/var/folders/g7/6fmhhq2x5813j_xlfxrqxln80000gn/T/ipykernel_13785/1439720932.py:6: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  import imp


In [1]:
import sys
sys.path.append('../../')
from src.utils import configuration, phonology
config = configuration.Config()

/Users/nicolewong/projects/child-grammar


# Extract IPA Annotated Words from Providence

In [3]:
pvd_idx = childespy.get_sql_query('select * from corpus where name = "Providence"',
                                 db_version = "2020.1").iloc[0]['id']

R[write to console]: Using supported database version: '2020.1'.



In [4]:
pvd_chi_tokens = childespy.get_sql_query('select gloss, target_child_name, target_child_age, \
speaker_code, actual_phonology, model_phonology, transcript_id, utterance_id, \
token_order from token where speaker_code = "CHI" and corpus_id = '+str(pvd_idx),
    db_version = "2020.1")
pvd_chi_tokens.to_csv(os.path.join(config.project_root,'output/csv/pvd_tokens.csv'), index=False)


R[write to console]: Using supported database version: '2020.1'.



In [5]:
excludes = ['*','(.)','(..)', '(...)','(....)','(.....)'] # these are either unlabeled, or marked for number of syllables (but not for phonology)
pvd_chi_tokens = pvd_chi_tokens.loc[~(pvd_chi_tokens.model_phonology.isin(excludes) |
    pvd_chi_tokens.actual_phonology.isin(excludes))]

In [6]:
pvd_chi_tokens.iloc[0]

gloss                   Mommy
target_child_name        Alex
target_child_age        514.0
speaker_code              CHI
actual_phonology          ɑmɪ
model_phonology         mɑmiː
transcript_id           42204
utterance_id         16759315
token_order                 1
Name: 1, dtype: object

In [7]:
# get the most common transcription for each form
pvd_chi_tokens.gloss = [x.lower() for x in pvd_chi_tokens.gloss]
ipa_for_glosses = pvd_chi_tokens.groupby(['gloss']).model_phonology.agg(lambda x:
    x.value_counts().reset_index().iloc[0]['index']    
).reset_index()

In [8]:
print(ipa_for_glosses.shape)
test = ipa_for_glosses.dropna()
test.shape

(7929, 2)


(7929, 2)

In [9]:
test_words = ['dog','cat','box','fishes', 'mommy', 'potato','a', 'read']
ipa_for_glosses.loc[ipa_for_glosses.gloss.isin(test_words)]

,gloss,model_phonology
4,a,ə
768,box,bɑks
1072,cat,kæt
1892,dog,dɑɡ
2420,fishes,fɪʃəz
4340,mommy,mɑmiː
5271,potato,pəteɪtoʊ
5507,read,ɹiːd


In [10]:
import imp
imp.reload(phonology)

<module 'src.utils.phonology' from '/Users/nicolewong/projects/child-grammar/src/tier_1/../../src/utils/phonology.py'>

In [11]:
ipa_for_glosses = ipa_for_glosses.loc[ipa_for_glosses.model_phonology != ""]
ipa_for_glosses['model_phonology'] = [phonology.strip_accents(phonology.phone_remap(x)) for x in ipa_for_glosses['model_phonology']]
ipa_for_glosses.shape


(6815, 2)

In [12]:
ipa_for_glosses.iloc[0:3]

,gloss,model_phonology
4,a,ə
14,abe,eəb
15,abide,əbaəd


Using just the model phonology from Providence is insufficient. So we map to citation forms in the CMU pronouncing dictionary.

# Link to CMU Citation Forms

In [13]:
cmu = pd.read_csv(os.path.join(config.project_root,'data/cmu_dict_df.csv'), encoding='latin1') # load an unedited version of the CMU pronunciation dictionary
cmu.columns = ['index','word','pronunciation']
from string import digits
remove_digits = str.maketrans('', '', digits)
cmu['phones'] = [[y.translate(remove_digits) for y in x.split(' ')] for x in cmu['pronunciation']] # remove indicators of stress marking 
cmu.word = [str(x).lower() for x in cmu.word]

In [14]:
# Output a CSV with all phonemes in the CMU (phon/phon_map.csv). Manually map these to phonemes and then load it (phon/phon_map_populated.csv)

# def flatten(list_of_lists):
#     return([item for subl in list_of_lists for item in subl])

# phone_inventory = pd.DataFrame({'arpa':np.unique(flatten(cmu.phones))})
# phone_inventory.to_csv('phon/phon_map.csv', index=False)


# Character differences / shorthand
# [X] cmu: ɝ -> əɹ
# [X] cmu:  tʃ -> ʧ
# [X] cmu  dʒ -> ʤ      
# Collapses
# [X] cmu  I -> schwa
# [X] ɑ in model phonlogy vs ɔ in cmu: convert ɔ to ɑ in both
# [X] model phonology: ʌ ->  ə

In [15]:
phone_map_df = pd.read_csv(os.path.join(config.project_root,'data/phon_map_populated.csv'))
phone_map = dict(zip(phone_map_df.arpa, phone_map_df.ipa))
cv_map = dict(zip(phone_map_df.arpa, phone_map_df.c_or_v))
cmu['ipa'] = [[phone_map[x] for x in y] for y in cmu.phones]
cmu['structure'] = [[cv_map[x] for x in y] for y in cmu.phones]
cmu['num_vowels'] = [np.sum(np.array(x) == 'v') for x in cmu['structure']]
cmu['ipa_short'] = [''.join(x) for x in cmu['ipa']]
cmu['ipa_short'] =  [x.replace('ɝ', 'əɹ').replace('ɪ','ə').replace('ɔ','ɑ').replace('ər', 'əɹ').replace('er','əɹ').replace('ʧ','tʃ').replace('ʤ','dʒ') for x in cmu['ipa_short']]
# remove any parenthetical + contents from the word record in CMU
cmu['word'] = [re.sub("\(.+\)$", '', x) for x in cmu['word']]

In [16]:
# confirm that we have multiple pronunciation entries for a word like "read"
cmu.loc[cmu.word=="read"]

,index,word,pronunciation,phones,ipa,structure,num_vowels,ipa_short
97825,97826,read,R EH1 D,"[R, EH, D]","[ɹ, ɛ, d]","[c, v, c]",1,ɹɛd
97826,97827,read,R IY1 D,"[R, IY, D]","[ɹ, i, d]","[c, v, c]",1,ɹid


In [17]:
cmu.loc[cmu.word.isin(test_words)]

,index,word,pronunciation,phones,ipa,structure,num_vowels,ipa_short
70,71,a,AH0,[AH],[ə],[v],1,ə
71,72,a,EY1,[EY],[eɪ],[v],1,eə
14024,14025,box,B AA1 K S,"[B, AA, K, S]","[b, ɑ, k, s]","[c, v, c, c]",1,bɑks
19019,19020,cat,K AE1 T,"[K, AE, T]","[k, æ, t]","[c, v, c]",1,kæt
33320,33321,dog,D AO1 G,"[D, AO, G]","[d, ɔ, ɡ]","[c, v, c]",1,dɑɡ
42452,42453,fishes,F IH1 SH AH0 Z,"[F, IH, SH, AH, Z]","[f, ɪ, ʃ, ə, z]","[c, v, c, v, c]",2,fəʃəz
42453,42454,fishes,F IH1 SH IH0 Z,"[F, IH, SH, IH, Z]","[f, ɪ, ʃ, ɪ, z]","[c, v, c, v, c]",2,fəʃəz
79787,79788,mommy,M AA1 M IY0,"[M, AA, M, IY]","[m, ɑ, m, i]","[c, v, c, v]",2,mɑmi
93614,93615,potato,P AH0 T EY1 T OW2,"[P, AH, T, EY, T, OW]","[p, ə, t, eɪ, t, oʊ]","[c, v, c, v, c, v]",3,pəteətoʊ
97825,97826,read,R EH1 D,"[R, EH, D]","[ɹ, ɛ, d]","[c, v, c]",1,ɹɛd


## Compute Proportion of Providence words that are in the adjusted CMU 

In [18]:
found_words = ipa_for_glosses.merge(cmu, left_on=['gloss'],
    right_on = ['word'])
found_words.drop_duplicates(subset=['word'])
# drop duplicates for a given word

found_words.shape[0] / ipa_for_glosses.shape[0]

0.953925165077036

Approximately 95% of the words in Providence are also present in CMU

In [19]:
# different pronunciations
missing_pronunctiations = ipa_for_glosses.loc[~ipa_for_glosses.gloss.isin(found_words.gloss)]
missing_pronunctiations_short = missing_pronunctiations[['gloss', 'model_phonology']]
missing_pronunctiations_short.sample(10)

,gloss,model_phonology
3174,hini,hini
3452,jappy,ʤæpi
2673,gasha,ɡɑʃə
7023,toad's,toʊdz
2818,goobadimunim,ɡubədəmjunəm
2647,gadaggers,ɡədæɡəɹz
619,blath,blæθ
4827,panacka,pænəkə
7501,wapsy,wɑpsi
5844,scribbly,skɹəbəli


These are justifiably odd words

## Compute Proportion of Providence annotations that match the annotation in adjusted CMU 

In [20]:
matched_words = ipa_for_glosses.merge(cmu, left_on=['gloss','model_phonology'],
    right_on = ['word','ipa_short'])
matched_words.shape[0] / ipa_for_glosses.shape[0]

0.8004402054292002

Approximately 86\% of words have the same pronunciation in the corrected CMU as in Providence

In [21]:
# different pronunciations
different_pronunctiations = ipa_for_glosses.loc[~ipa_for_glosses.gloss.isin(matched_words.gloss)]
different_pronunctiations = different_pronunctiations.merge(cmu, left_on=['gloss'], right_on=['word'])
different_pronunctiations_short = different_pronunctiations[['gloss', 'model_phonology','ipa_short']]
different_pronunctiations_short.columns = ['gloss','pvd_ipa','cmu_ipa']
different_pronunctiations_short.sample(10)

,gloss,pvd_ipa,cmu_ipa
284,jackets,ʤækəts,dʒækəts
131,chicks,ʧəks,tʃəks
595,vulture,vəlʧəɹ,vəltʃəɹ
110,cheeseburger,ʧizbəɹɡəɹ,tʃizbəɹɡəɹ
535,singed,sənʤd,səndʒd
68,catch,kæʧ,kætʃ
39,berlioz,bɛɹlioʊz,bəɹlioʊz
531,sergeant,sɑɹʤənt,sɑɹdʒənt
118,chester,ʧɛstəɹ,tʃɛstəɹ
103,cheeks,ʧiks,tʃiks


In [22]:
childes_counts = pd.read_csv(os.path.join(config.project_root, 'data/vocab.csv'))
childes_counts.columns 
cmu_in_childes = cmu.loc[(cmu.word.isin(
    childes_counts.loc[childes_counts['count'] > 3].word)) & (cmu['num_vowels'] <=2)]

In [23]:
cmu_in_childes.shape

(13894, 8)

# Compare with Coverage in the BERT Tokenizer

In [24]:
# there's a circularity here -- we need a BERT model in order to know what all is in the tokenizer

In [25]:
from transformers import BertTokenizer
initial_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


/Users/nicolewong/opt/anaconda3/envs/child_grammar/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Downloading: 100%|███████████████████████████| 232k/232k [00:00<00:00, 13.5MB/s]
Downloading: 100%|███████████████████████████| 28.0/28.0 [00:00<00:00, 10.6kB/s]
Downloading: 100%|██████████████████████████████| 570/570 [00:00<00:00, 232kB/s]


In [26]:
bert_tokenizer_vocab = initial_tokenizer.get_vocab().keys()

In [27]:
cmu_in_childes.shape

(13894, 8)

In [28]:
cmu_in_childes_with_tokenizer = cmu.loc[(cmu.word.isin(
    childes_counts.loc[childes_counts['count'] > 3].word)) & (cmu['num_vowels'] <=2) &
            (cmu.word.isin(bert_tokenizer_vocab))]
cmu_in_childes_with_tokenizer.shape

(8943, 8)

In [29]:
cmu_in_childes_with_tokenizer.to_csv(os.path.join(config.project_root,'data/cmu_in_childes.csv'),index=False)
cmu_in_childes_with_tokenizer.to_pickle(os.path.join(config.project_root,'output/pkl/cmu_in_childes.pkl'))


In [30]:
cmu_in_childes_with_tokenizer.loc[cmu_in_childes_with_tokenizer.word =='read']

,index,word,pronunciation,phones,ipa,structure,num_vowels,ipa_short
97825,97826,read,R EH1 D,"[R, EH, D]","[ɹ, ɛ, d]","[c, v, c]",1,ɹɛd
97826,97827,read,R IY1 D,"[R, IY, D]","[ɹ, i, d]","[c, v, c]",1,ɹid


In [31]:
cmu_in_childes_with_tokenizer.loc[cmu_in_childes_with_tokenizer.word == 'watch']

,index,word,pronunciation,phones,ipa,structure,num_vowels,ipa_short
128558,128559,watch,W AA1 CH,"[W, AA, CH]","[w, ɑ, ʧ]","[c, v, c]",1,wɑtʃ
128559,128560,watch,W AO1 CH,"[W, AO, CH]","[w, ɔ, ʧ]","[c, v, c]",1,wɑtʃ


# Compute Coverage of MacArthur CDI words

In [32]:
wswg  = pd.read_csv(os.path.join(config.project_root, 'data/WSWG_50percentproducing.csv')) # load a list of all Words and Sentences + Words and Gestures words

In [33]:
wswg['word'] = [x.split(' ')[0].split('/')[0] for x in wswg.word]
' '.join(wswg['word'])
wswg_cmu = wswg.merge(cmu_in_childes)
wswg_cmu = wswg_cmu.drop_duplicates(subset=['word'])
print('Total coverage of MacArthur CDI words, before intersecting with BERT tokenizer')
print(wswg_cmu.shape[0] / wswg.shape[0])


Total coverage of MacArthur CDI words, before intersecting with BERT tokenizer
0.8918495297805643


In [34]:
wswg['word'] = [x.split(' ')[0].split('/')[0] for x in wswg.word]
' '.join(wswg['word'])
wswg_cmu = wswg.merge(cmu_in_childes_with_tokenizer)
wswg_cmu = wswg_cmu.drop_duplicates(subset=['word'])
print('Total coverage of MacArthur CDI words, after intersecting with BERT tokenizer')
print(wswg_cmu.shape[0] / wswg.shape[0])

Total coverage of MacArthur CDI words, after intersecting with BERT tokenizer
0.8197492163009404


In [35]:
# What words from the CDI are missing? 
' '.join(wswg.loc[~wswg.word.isin(wswg_cmu.word)].word)

"woof yum grrr moo meow banana vroom ouch quack baa shh cracker peekaboo owie choo diaper child's telephone potty pet's tummy toothbrush bicycle TV yucky cereal bathtub buttocks elephant butterfly crayon teddybear tickle pajamas cheerios yogurt bib carrots I raisin strawberry giraffe firetruck medicine pancake animal potato spaghetti hamburger helicopter babysitter's cockadoodledoo stroller potato motorcycle applesauce alligator pattycake popsicle crib refrigerator pretzel don't pickle muffin lemme donut slipper snowman radio vitamins sandbox lollipop underpants mittens fireman dryer jello melon another mailman sprinkler tricycle tomorrow mop hafta sneaker sled babysitter"

We are missing multisyllable words, onomatopoeia from CHILDES